In [1]:
#pip install pymupdf pdfplumber pandas


Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/16.6 MB ? eta -:--:--
   ----- ---------------------------------- 2.4/16.6 MB 12.1 MB/s eta 0:00:02
   ---------- ----------------------------- 4.2/16.6 MB 10.7 MB/s eta 0:00:02
   ------------- -------------------------- 5.5/16.6 MB 8.9 MB/s eta 0:00:02
   ---------------- ----------------------- 6.8/16.6 MB 8.2 MB/s eta 0:00:02
   ------------------- -------------------- 8.1/16.6 MB 7.7 MB/s eta 0:00:02
   ---------------------- ----------------- 9.2/16.6 MB 7.4 MB/s eta 0:00:01
   ------------------------- -------------- 10.5/16.6 MB 7.2 MB/s eta 0:00:01
   ---------------------------- ----------- 11.8/16.6 MB 7.1 MB/s eta 0:00:01
   ------------------------------- -------- 13.1/16.6 MB 7.0 MB/s eta 0:00:01
   ---------------------------------- ----- 14.2/16.6 MB 6.9 MB/s eta 0:00:01
   ------------------------------------- -- 15.5/16.6 MB 6.8 MB/s eta 0:00:0

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


 Step 1: Import Required Libraries


In [29]:
import os
import fitz  # PyMuPDF
import pdfplumber
import pandas as pd
import warnings
import warnings
warnings.filterwarnings("ignore", message="CropBox missing from /Page, defaulting to MediaBox")



Step 2: Setup File Paths

In [30]:
# Define folder path
pdf_folder = r"C:\Users\FINRISE\Desktop\Task data scie\case_study_FTE\case_study_FTE\case_study_1\data\section_two_data"


In [31]:
pdf_folder

'C:\\Users\\FINRISE\\Desktop\\Task data scie\\case_study_FTE\\case_study_FTE\\case_study_1\\data\\section_two_data'

In [32]:
# List all PDF files
pdf_files = [os.path.join(pdf_folder, file) for file in os.listdir(pdf_folder) if file.endswith('.pdf')]


In [33]:
pdf_files

['C:\\Users\\FINRISE\\Desktop\\Task data scie\\case_study_FTE\\case_study_FTE\\case_study_1\\data\\section_two_data\\CYBER SECURITY (R18A0521).pdf',
 'C:\\Users\\FINRISE\\Desktop\\Task data scie\\case_study_FTE\\case_study_FTE\\case_study_1\\data\\section_two_data\\Cyber Security Essentials ( PDFDrive ).pdf',
 'C:\\Users\\FINRISE\\Desktop\\Task data scie\\case_study_FTE\\case_study_FTE\\case_study_1\\data\\section_two_data\\CyberCrimes-CommonTypes.pdf',
 'C:\\Users\\FINRISE\\Desktop\\Task data scie\\case_study_FTE\\case_study_FTE\\case_study_1\\data\\section_two_data\\cybersecuirty_sb_factsheets_all.pdf',
 'C:\\Users\\FINRISE\\Desktop\\Task data scie\\case_study_FTE\\case_study_FTE\\case_study_1\\data\\section_two_data\\CyberSecurityConclaveAtVigyanBhavanDelhi_1.pdf',
 'C:\\Users\\FINRISE\\Desktop\\Task data scie\\case_study_FTE\\case_study_FTE\\case_study_1\\data\\section_two_data\\Introduction-cyber-security.pdf']

Step 3: Extract All Text (Master DataFrame)


In [34]:
def extract_text_from_pdfs(pdf_files):
    records = []
    for file in pdf_files:
        doc = fitz.open(file)
        for page_num, page in enumerate(doc, start=1):
            text = page.get_text()
            records.append({
                'file': os.path.basename(file),
                'page': page_num,
                'text': text
            })
        doc.close()
    return pd.DataFrame(records)

# Call the function
master_dataframe = extract_text_from_pdfs(pdf_files)
master_dataframe.head()


,file,page,text
0,CYBER SECURITY (R18A0521).pdf,1,CYBER SECURITY \nPage 1 \n \nDIGITAL NOTES \nO...
1,CYBER SECURITY (R18A0521).pdf,2,CYBER SECURITY \nPage 2 \n \nMALLA REDDY COLLE...
2,CYBER SECURITY (R18A0521).pdf,3,CYBER SECURITY \nPage 3 \n \nUNIT - V \nPrivac...
3,CYBER SECURITY (R18A0521).pdf,4,CYBER SECURITY \nPage 4 \n \n ...
4,CYBER SECURITY (R18A0521).pdf,5,CYBER SECURITY \nPage 5 \n \nUNIT-I \nIntroduc...


Markdown: This step extracts plain text from every page of each PDF and creates a unified DataFrame.



Step 4: Extract Bold Text

In [35]:
def extract_bold_text(pdf_files):
    bold_records = []
    for file in pdf_files:
        doc = fitz.open(file)
        for page_num, page in enumerate(doc, start=1):
            blocks = page.get_text("dict")["blocks"]
            bold_text = ""
            for b in blocks:
                for l in b.get("lines", []):
                    for s in l.get("spans", []):
                        if "bold" in s["font"].lower():
                            bold_text += s["text"] + " "
            bold_records.append({
                'file': os.path.basename(file),
                'page': page_num,
                'bold_text': bold_text.strip()
            })
        doc.close()
    return pd.DataFrame(bold_records)

# Call the function
master_dataframe_two = extract_bold_text(pdf_files)
master_dataframe_two.head()


,file,page,bold_text
0,CYBER SECURITY (R18A0521).pdf,1,DIGITAL NOTES ON CYBER SECURITY (R18A0521) ...
1,CYBER SECURITY (R18A0521).pdf,2,MALLA REDDY COLLEGE OF ENGINE ERING AND TECHNO...
2,CYBER SECURITY (R18A0521).pdf,3,UNIT - V Privacy Issues: Cybercrime: Example...
3,CYBER SECURITY (R18A0521).pdf,4,MALLA REDDY COLLEGE OF ENGINEERING & TECHNOLOG...
4,CYBER SECURITY (R18A0521).pdf,5,UNIT-I Introduction to Cyber Security Cyber ...


Markdown: This function iterates through each span and checks if the font contains "bold" to extract bold text.

Step 5: Extract Underlined Text


In [36]:
def extract_underlined_text(pdf_files):
    underline_records = []
    for file in pdf_files:
        doc = fitz.open(file)
        for page_num, page in enumerate(doc, start=1):
            blocks = page.get_text("dict")["blocks"]
            underlined_text = ""
            for b in blocks:
                for l in b.get("lines", []):
                    for s in l.get("spans", []):
                        if s.get("underline", 0):  # underline = 1
                            underlined_text += s["text"] + " "
            underline_records.append({
                'file': os.path.basename(file),
                'page': page_num,
                'underlined_text': underlined_text.strip()
            })
        doc.close()
    return pd.DataFrame(underline_records)

# Call the function
master_dataframe_three = extract_underlined_text(pdf_files)
master_dataframe_three.head()


,file,page,underlined_text
0,CYBER SECURITY (R18A0521).pdf,1,
1,CYBER SECURITY (R18A0521).pdf,2,
2,CYBER SECURITY (R18A0521).pdf,3,
3,CYBER SECURITY (R18A0521).pdf,4,
4,CYBER SECURITY (R18A0521).pdf,5,


Handle Empty Tables Gracefully


In [37]:
def extract_text_from_pdfs(pdf_files):
    """
    Extract plain text from each page of all PDF files.
    Returns a DataFrame with file name, page number, and extracted text.
    """
    records = []
    for file in pdf_files:
        doc = fitz.open(file)
        for page_num, page in enumerate(doc, start=1):
            text = page.get_text()
            records.append({
                'file': os.path.basename(file),
                'page': page_num,
                'text': text
            })
        doc.close()
    return pd.DataFrame(records)


Check if Tables Were Found


In [42]:
print(f"Total tables found: {len(table_data_list)}")


Total tables found: 492


Check Extracted Tables from All PDF

In [43]:
# Extract tables
table_data_list = extract_tables_from_pdfs(pdf_files)

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, def

This will help you verify if the extracted tables look as expected or if any are misaligned or empty.



In [45]:
#You can inspect them with this:


for i, record in enumerate(table_data_list):
    print(f"\nTable #{i+1} from file: {record['file']} (Page {record['page']})")
    display(record['table_data'].head())  # or use print(record['table_data']) if not in Jupyter



Table #1 from file: CYBER SECURITY (R18A0521).pdf (Page 4)


,0,1,2,3
0,S. No,Unit,Topic,Page no
1,1,I,Cyber security introduction -Basics,5
2,2,I,Layers of Security,9
3,3,I,"Security vulnerabilities, threats and Attacks",11
4,4,I,Cyber Threats-Cyber-Warfare,16



Table #2 from file: CYBER SECURITY (R18A0521).pdf (Page 5)


,0
0,Cyber security is the most concerned matter as...
1,Attackers are now using more sophisticated tec...
2,"small-scale businesses or large organization, ..."
3,whether IT or non-IT firms have understood the...
4,on adopting all possible measures to deal with...



Table #3 from file: CYBER SECURITY (R18A0521).pdf (Page 5)


...

,0
0,If your phone is rooted you may consider encry...
1,volume on the Smartphone to protect some infor...
2,"Luks Manager allows easy, on-the-fly strong en..."
3,interface. We highly recommend that you instal...
4,data on your Android device and use the Encryp...



Table #491 from file: Introduction-cyber-security.pdf (Page 143)


,0
0,A VPN provides an encrypted tunnel through the...
1,"server. This is called a tunnel, because unlik..."
2,"services, protocols, and contents. A VPN conne..."
3,you decide.
4,Note that since all your traffic goes through ...



Table #492 from file: Introduction-cyber-security.pdf (Page 143)


,0
0,https://www.youtube.com/watch?v=KCObM4PBTVk
1,https://www.youtube.com/watch?v=V8bCyRCqK0k


In [46]:
# Convert to readable master dataframe
master_dataframe_four = pd.DataFrame([
    {
        'file': rec['file'],
        'page': rec['page'],
        'table': rec['table_data'].to_string(index=False, header=False)
    }
    for rec in table_data_list
])

In [47]:
master_dataframe_four.head()


,file,page,table
0,CYBER SECURITY (R18A0521).pdf,4,S. No Unit ...
1,CYBER SECURITY (R18A0521).pdf,5,Cyber security is the most concerned matter as...
2,CYBER SECURITY (R18A0521).pdf,5,The term cyber security refers to techniques a...
3,CYBER SECURITY (R18A0521).pdf,5,Cyber security is the protection of Internet-c...
4,CYBER SECURITY (R18A0521).pdf,5,Cyber is related to the technology which conta...


In [54]:
# Define output directory
output_dir = r"C:\Users\FINRISE\Desktop\Task data scie\case_study_FTE\case_study_FTE\case_study_1\data\section_two_data"


In [55]:
output_dir

'C:\\Users\\FINRISE\\Desktop\\Task data scie\\case_study_FTE\\case_study_FTE\\case_study_1\\data\\section_two_data'

In [59]:
# Save the first table as a CSV
if table_data_list:
    output_path = os.path.join(output_dir, "sample_table.csv")
    table_data_list[0]['table_data'].to_csv(output_path, index=False, header=False)
    print(f"Table saved at: {output_path}")
else:
    print("No table data found.")


Table saved at: C:\Users\FINRISE\Desktop\Task data scie\case_study_FTE\case_study_FTE\case_study_1\data\section_two_data\sample_table.csv
